<a href="https://colab.research.google.com/github/Javigomez4/skills-introduction-to-github/blob/main/ITF6575_Assignment1_JavierGomezAguilar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ITF6575: Methods of Operations Research  
Assignment 1
Instructor: Utsav Sadana   
Student: Javier Gomez Aguilar  
Date: 16 October 2025  
  



## Problem 3 – Simplex Tableau Sensitivity Analysis

In [1]:

import numpy as np

alpha = 0
beta = 1
gamma = 0

rhs = np.array([0.5, 1])

if beta > 0 and gamma <= 0:
    print("Dual is infeasible (primal unbounded).")
else:
    print("Dual may be feasible.")

alpha_values = np.linspace(-5, 5, 11)
print("\nSensitivity Analysis:")
for eps in alpha_values:
    r1 = alpha + eps
    if r1 <= 0:
        print(f"ε = {eps:>5.2f}: Basis remains optimal (r1={r1})")
    else:
        print(f"ε = {eps:>5.2f}: Basis no longer optimal (r1={r1})")


Dual is infeasible (primal unbounded).

Sensitivity Analysis:
ε = -5.00: Basis remains optimal (r1=-5.0)
ε = -4.00: Basis remains optimal (r1=-4.0)
ε = -3.00: Basis remains optimal (r1=-3.0)
ε = -2.00: Basis remains optimal (r1=-2.0)
ε = -1.00: Basis remains optimal (r1=-1.0)
ε =  0.00: Basis remains optimal (r1=0.0)
ε =  1.00: Basis no longer optimal (r1=1.0)
ε =  2.00: Basis no longer optimal (r1=2.0)
ε =  3.00: Basis no longer optimal (r1=3.0)
ε =  4.00: Basis no longer optimal (r1=4.0)
ε =  5.00: Basis no longer optimal (r1=5.0)


## Problem 4 – Cutting Stock Problem


We are given **K = 20 rolls** of width **W = 100 inches**.  
The goal is to minimize the number of rolls needed to satisfy demand, allowing trim loss.

| Width (in) | 40 | 20 | 10 |
|-------------|----|----|----|
| Demand      | 8  | 7  | 10 |

We solve this in four parts:
1. Enumerate all feasible patterns  
2. Solve the LP relaxation  
3. Solve the integer version  
4. Run column generation to verify the LP result


### Step 1 – Enumerate Feasible Patterns

In [2]:

import numpy as np

W = 100
Wa, Wb, Wc = 40, 20, 10

patterns = []
for a in range(W // Wa + 1):
    for b in range(W // Wb + 1):
        for c in range(W // Wc + 1):
            used = a*Wa + b*Wb + c*Wc
            if used <= W:
                waste = W - used
                patterns.append((a, b, c, waste))

patterns = np.array(patterns)
print(f"Total feasible patterns: {len(patterns)}")
print("Example patterns (first 10):")
print(patterns[:10])


Total feasible patterns: 56
Example patterns (first 10):
[[  0   0   0 100]
 [  0   0   1  90]
 [  0   0   2  80]
 [  0   0   3  70]
 [  0   0   4  60]
 [  0   0   5  50]
 [  0   0   6  40]
 [  0   0   7  30]
 [  0   0   8  20]
 [  0   0   9  10]]


### Step 2 – LP Relaxation (Fractional Solution)

In [3]:

!pip install pulp

import pulp

widths = [40, 20, 10]
demand = [8, 7, 10]

model = pulp.LpProblem("CuttingStock_LP", pulp.LpMinimize)
z = [pulp.LpVariable(f"z_{i}", lowBound=0) for i in range(len(patterns))]

model += pulp.lpSum(z)
for i in range(3):
    model += pulp.lpSum(patterns[p][i] * z[p] for p in range(len(patterns))) >= demand[i]

model.solve(pulp.PULP_CBC_CMD(msg=0))

print("\nLP Relaxation Solution:")
for p in range(len(patterns)):
    if z[p].value() and z[p].value() > 0:
        print(f"Pattern {patterns[p][:3]} -> {z[p].value():.2f} rolls")

print(f"\nLP optimal number of rolls: {pulp.value(model.objective):.2f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 69.5 MB/s eta 0:00:00

LP Relaxation Solution:
Pattern [ 0  0 10] -> 1.00 rolls
Pattern [0 5 0] -> 0.60 rolls
Pattern [2 1 0] -> 4.00 rolls

LP optimal number of rolls: 5.60


### Step 3 – Integer Solution (Exact Minimum Rolls)

In [4]:

# Integer version using PuLP
int_model = pulp.LpProblem("CuttingStock_Integer", pulp.LpMinimize)
x = [pulp.LpVariable(f"x_{i}", lowBound=0, cat="Integer") for i in range(len(patterns))]

int_model += pulp.lpSum(x)
for i in range(3):
    int_model += pulp.lpSum(patterns[p][i] * x[p] for p in range(len(patterns))) >= demand[i]

int_model.solve(pulp.PULP_CBC_CMD(msg=0))

print("\nInteger Optimal Solution:")
for p in range(len(patterns)):
    if x[p].value() and x[p].value() > 0:
        print(f"Pattern {patterns[p][:3]} -> {int(x[p].value())} rolls")

print(f"\nInteger optimal number of rolls: {int(pulp.value(int_model.objective))}")



Integer Optimal Solution:
Pattern [ 0  0 10] -> 1 rolls
Pattern [0 5 0] -> 1 rolls
Pattern [2 1 0] -> 4 rolls

Integer optimal number of rolls: 6


### Step 4 – Column Generation (LP Verification)

In [5]:

# Column generation implementation
TOL = 1e-6
MAX_ITERS = 20

def initial_patterns(W, widths):
    pats = []
    for i, wi in enumerate(widths):
        patt = [0]*len(widths)
        patt[i] = W // wi
        pats.append(patt)
    return pats

def solve_master(patterns, widths, demand):
    m = pulp.LpProblem("MasterLP", pulp.LpMinimize)
    x = [pulp.LpVariable(f"x_{j}", lowBound=0) for j in range(len(patterns))]
    m += pulp.lpSum(x)
    constraints = []
    for i in range(len(widths)):
        cons = pulp.lpSum(patterns[j][i]*x[j] for j in range(len(patterns))) >= demand[i]
        cname = f"cover_{i}"
        m += cons, cname
        constraints.append(m.constraints[cname])
    m.solve(pulp.PULP_CBC_CMD(msg=0))
    obj = pulp.value(m.objective)
    duals = [c.pi for c in constraints]
    return obj, duals

def pricing_knapsack(W, widths, duals):
    dp = [0]*(W+1)
    choice = [-1]*(W+1)
    for c in range(1, W+1):
        for i, wi in enumerate(widths):
            if wi <= c and dp[c-wi] + duals[i] > dp[c] + TOL:
                dp[c] = dp[c-wi] + duals[i]
                choice[c] = i
    best_cap = np.argmax(dp)
    counts = [0]*len(widths)
    c = best_cap
    while c > 0 and choice[c] != -1:
        i = choice[c]
        counts[i] += 1
        c -= widths[i]
    return dp[best_cap], counts

patterns = initial_patterns(W, widths)
for it in range(1, MAX_ITERS+1):
    obj, duals = solve_master(patterns, widths, demand)
    best_val, new_pattern = pricing_knapsack(W, widths, duals)
    reduced_cost = 1 - best_val
    print(f"Iter {it}: Obj = {obj:.4f}, Duals = {[round(d,3) for d in duals]}, Reduced cost = {reduced_cost:.4f}")
    if reduced_cost < -TOL:
        patterns.append(new_pattern)
        print(f"  Added new pattern: {new_pattern}")
    else:
        print("  No improving pattern found. Converged.")
        break

print("\nFinal LP Objective (approx.):", round(obj,4))
print("Final patterns used:")
for p in patterns:
    print(" ", p)


Iter 1: Obj = 6.4000, Duals = [0.5, 0.2, 0.1], Reduced cost = -0.2000
  Added new pattern: [2, 1, 0]
Iter 2: Obj = 5.6000, Duals = [0.4, 0.2, 0.1], Reduced cost = 0.0000
  No improving pattern found. Converged.

Final LP Objective (approx.): 5.6
Final patterns used:
  [2, 0, 0]
  [0, 5, 0]
  [0, 0, 10]
  [2, 1, 0]



Column Generation Summary
Column generation starts with simple patterns (each cutting only one type of width).  
It solves the LP (master problem) and then generates new patterns using the dual prices  
until no improving pattern can be found (reduced cost ≥ 0).

This process gives the same fractional optimum as the LP relaxation, **zLP = 5.6**,  
which is smaller than the integer optimum **zINT = 6**, showing that the LP relaxation is not integral.
Final Results
| Method | Description | Optimum (rolls) |
|---------|--------------|-----------------|
| Enumeration | Full pattern list | 56 patterns found |
| LP Relaxation | Fractional rolls allowed | **5.6** |
| Integer Solution | Whole rolls only | **6** |
| Column Generation | LP verification | **5.6** |
